In [33]:
from __future__ import annotations
from urllib.request import urlopen 
from xml.etree import ElementTree as ET 
from functools import lru_cache 
from typing import Optional 
from datetime import timedelta, datetime 
import time

def get_currencies(currencies_ids_lst=['R01235', 'R01239', 'R01820']):
    """
    получение курсов валют с сайта Центробанка
    """
    cur_res_str = urlopen("http://www.cbr.ru/scripts/XML_daily.asp")
    result = {}
    cur_res_xml = ET.parse(cur_res_str)
    root = cur_res_xml.getroot()
    valutes = root.findall('Valute')
    for el in valutes:
        valute_id = el.get('ID')
        if str(valute_id) in currencies_ids_lst:
            valute_cur_val = el.find('Value').text
            result[valute_id] = valute_cur_val
    time.process_time()
    return result

class SingletonMeta(type): 

    _instance: Optional[Singleton] = None 
    def __call__(self) -> Singleton: 
        if self._instance is None: 
            self._instance = super().__call__() 
        return self._instance 


class CurrencyBoard(metaclass=SingletonMeta):
    """
    класс-синглтон
    """
    
    def __init__(self):
            self.currencies = ['R01235','R01239','R01820']
            self.rates = get_currencies(self.currencies)

    def get_from_cache(self, code):
        """
        без запроса к сайту
        """
        return self.currencies[code]

    def get_new_currency(self, code):
        """
        запрос информации о курсах новой валюты  и добавление её в кэш
        """
        self.currencies.append(code)
        self.rates.update(get_currencies([code]))
        return self.rates[code]

    def update(self):
        """
        принудительно обновляем данных
        """
        new = get_currencies(self.currencies)
        self.rates.update(dict(zip(sorted(self.currencies),new.values())))
        return self.rates

    def check(self):
        """
        проводим проверку времени обновления
        """

        if (time.process_time() > 5*60):
            return get_currencies(self.currencies)
        else:
            print('Обновление не требуется')

cur_vals = get_currencies()

print("\ndollar = USD = R01235 \neuro = EUR = R01239 \niena = GBP = R01820  \n", cur_vals)


dollar = USD = R01235 
euro = EUR = R01239 
iena = GBP = R01820  
 {'R01235': '62,5229', 'R01239': '71,2323', 'R01820': '58,4190'}
